In [46]:
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Times']})
rc('text', usetex=True)
import warnings
warnings.filterwarnings('ignore')

import pathlib
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import gala.dynamics as gd
import gala.potential as gp
import jax
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import torusimaging as oti
from astropy.constants import G
from gala.units import galactic
from pyia import GaiaData
from astropy.io import fits 
import tqdm
import cmasher as cm
import os
import model_radial_old as model_radial
import astropy.units as u
from scipy import stats
jax.config.update("jax_enable_x64", True)

In [47]:
# LOAD THE DATA
path = '/Users/dhortad/Projects/TorusImaging-radial/data/qiso_df_sim_Rz.fits'
tb = fits.open(path)
data = tb[1].data 

print(len(data))

10000000


In [48]:
# Eilers rotation curve
Rgal = data['R']*u.kpc
Rg = data['Rg']*u.kpc
lz = data['J'][:,1]*u.kpc**2/u.Myr
lz = lz.to((u.kpc*u.km)/u.s)
vcirc = 229. *u.km/u.s
deltaR = Rgal-Rg

#transform to cylindrical velocities
vR = data['v_R']*u.kpc/u.Myr

In [49]:
jr = data['J'][:,0]*u.kpc**2/u.Myr
jr = jr.to(u.km/u.s * u.kpc)

In [50]:
# use APW's function to paint on abundances given correlation with Jr
def make_mgfe(JR, slope=0.1 / 10, std=0.05, rng=None):
    """
    Default parameters from APOGEE in fiducial model
    """
    if rng is None:
        rng = np.random.default_rng()

    x = np.sqrt(JR)
    mgfe = slope * x + 0.0
    mgfe = rng.normal(mgfe, std)
    mgfe_err = np.exp(rng.normal(-4.0, 0.5, size=len(JR)))
    mgfe = rng.normal(mgfe, mgfe_err)
    return mgfe, mgfe_err

lab, lab_err = make_mgfe(jr)

In [51]:
from astropy.stats import bootstrap
# we will bootstrap 10 times for now to speed things up
def create_boot_samp(deltaR, vR,xfe,xfe_err,bins,n=10):
    deltaR_boot = []
    vRs_boot = []
    xfe_boot = []
    xfe_err_boot = []
    
    #loop over bins
    for indx, i in enumerate(bins):
        # create the bootstrap samples  
        # in order to sample the distribution using bootstrap with resampling and to get the correct
        # information for every star picked, we need to sample an id array, and use that to pick out the sampled stars
        # otherwise you get incorrect fehs for every abundance
        # random_ids = np.arange(len(deltaR[i]))
        random_ids = np.random.choice(len(deltaR[i]), size=50000)
        samples_indices = bootstrap(random_ids, n).astype(int)
        # within a bin, loop over all the 
        deltaR_bo = []
        vRs_bo = []
        xfe_bo = []
        xfe_err_bo = []
        for jndx, j in enumerate(samples_indices):
            # find the stars with the correct id
            deltaR_b = deltaR[i][j]
            deltaR_bo.append(deltaR_b)
            vRs_b = vR[i][j]
            vRs_bo.append(vRs_b)
            xfe_b = xfe[i][j]
            xfe_bo.append(xfe_b)
            xfe_err_b = xfe_err[i][j]
            xfe_err_bo.append(xfe_err_b)
            
        deltaR_boot.append(deltaR_bo)
        vRs_boot.append(vRs_bo)
        xfe_boot.append(xfe_bo)
        xfe_err_boot.append(xfe_err_bo)
            
    return deltaR_boot, vRs_boot, xfe_boot,xfe_err_boot

In [52]:
Rg_w = 1.
Rgal_cs = np.linspace(5,12,15)

bins = []
for Rg_c in Rgal_cs:
    Rg_l, Rg_r = (Rg_c, Rg_c + Rg_w)
    Rg_mask = (Rg.value > Rg_l) & (Rg.value <= Rg_r) & (np.abs(data['z'])<0.2) & (data['v_z']<0.03) &(jr.value<200) 
    bins.append(Rg_mask)

In [44]:
deltaR_boot, vRs_boot, xfe_boot, xfe_err_boot = create_boot_samp(deltaR.value, vR.value,lab,lab_err,bins,n=10)


In [45]:
np.save('../sav/deltaR_boot', deltaR_boot)
np.save('../sav/vRs_boot', vRs_boot)
np.save('../sav/xfe_boot', xfe_boot)
np.save('../sav/xfe_err_boot', xfe_err_boot)

# now assume wrong rotation curve

In [53]:
# Eilers rotation curve
Rgal = data['R']*u.kpc
lz = data['J'][:,1]*u.kpc**2/u.Myr
lz = lz.to((u.kpc*u.km)/u.s)
vcirc_w = 210. *u.km/u.s
Rg_w = lz/vcirc_w
deltaR_w = Rgal-Rg_w

#transform to cylindrical velocities
vR = data['v_R']*u.kpc/u.Myr

In [54]:
deltaR_boot_w, vRs_boot_w, xfe_boot_w, xfe_err_boot_w = create_boot_samp(deltaR_w.value, vR.value,lab,lab_err,bins,n=10)


In [55]:
np.save('../sav/deltaR_boot_w', deltaR_boot_w)
np.save('../sav/vRs_boot_w', vRs_boot_w)
np.save('../sav/xfe_boot_w', xfe_boot_w)
np.save('../sav/xfe_err_boot_w', xfe_err_boot_w)